<a href="https://colab.research.google.com/github/kapilnchauhan77/Disfluency-Detection/blob/main/Disfluency_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!sudo apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.0 MB/s 
     |████████████████████████████████| 452 kB 70.5 MB/s 
     |████████████████████████████████| 182 kB 68.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.4 MB/s 
     |████████████████████████████████| 213 kB 70.0 MB/s 
     |████████████████████████████████| 132 kB 68.7 MB/s 
     |████████████████████████████████| 127 kB 59.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 n

# Finetune

In [29]:
from datasets import load_dataset
dataset = load_dataset("disfl_qa")

  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, BertModel

model = TFAutoModelForSequenceClassification.from_pretrained("kapilchauhan/pretrained-bert-free_speech", num_labels=2)
# model = BertModel.from_pretrained("kapilchauhan/pretrained-bert-free_speech", from_tf=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("kapilchauhan/pretrained-bert-free_speech")

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [30]:
import pandas as pd

df_train = dataset['train'].to_pandas()
df_train

original_df = pd.DataFrame()
original_df['text'] = df_train['original question']
original_df['labels'] = 0

disfluency_df = pd.DataFrame()
disfluency_df['text'] = df_train['disfluent question']
disfluency_df['labels'] = 1

train_dataset = disfluency_df.append(original_df, ignore_index=True)

df_test = dataset['test'].to_pandas()
df_test

original_df = pd.DataFrame()
original_df['text'] = df_test['original question']
original_df['labels'] = 0

disfluency_df = pd.DataFrame()
disfluency_df['text'] = df_test['disfluent question']
disfluency_df['labels'] = 1

test_dataset = disfluency_df.append(original_df, ignore_index=True)

df_validation = dataset['validation'].to_pandas()
df_validation

original_df = pd.DataFrame()
original_df['text'] = df_validation['original question']
original_df['labels'] = 0

disfluency_df = pd.DataFrame()
disfluency_df['text'] = df_validation['disfluent question']
disfluency_df['labels'] = 1

validation_dataset = disfluency_df.append(original_df, ignore_index=True)

In [31]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict()

tds = Dataset.from_pandas(train_dataset)
tsds = Dataset.from_pandas(test_dataset)
vds = Dataset.from_pandas(validation_dataset)

dataset['train'] = tds
dataset['test'] = tsds
dataset['validation'] = vds

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 14364
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 7286
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 2000
    })
})

In [32]:
from transformers import AutoTokenizer
import multiprocessing

num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)

The max length for the tokenizer is: 512
    

#0:   0%|          | 0/8 [00:00<?, ?ba/s]

#1:   0%|          | 0/8 [00:00<?, ?ba/s]

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
tokenized_datasets['train']

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 14364
})

In [35]:
from transformers import AutoTokenizer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
1796/1796 [==============================] - 190s 95ms/step - loss: 0.1105 - sparse_categorical_accuracy: 0.9694 - val_loss: 0.0821 - val_sparse_categorical_accuracy: 0.9800
Epoch 2/3
1796/1796 [==============================] - 167s 93ms/step - loss: 0.0942 - sparse_categorical_accuracy: 0.9759 - val_loss: 0.0987 - val_sparse_categorical_accuracy: 0.9765
Epoch 3/3
1796/1796 [==============================] - 166s 92ms/step - loss: 0.0814 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.0816 - val_sparse_categorical_accuracy: 0.9795
